In [ ]:
import boto3
import os

def download_s3_bucket(bucket_name, local_dir):
    # AWS credentials
    s3 = boto3.client(
        's3',
        aws_access_key_id=' ',
        aws_secret_access_key=' ',
        region_name=' '  # Replace with your region
    )

      # Ensure local directory exists
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)

    # List all objects in the S3 bucket
    paginator = s3.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name)

    # Iterate over the objects and download each file
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                s3_file_path = obj['Key']

                # Skip "folder" keys that end with '/'
                if s3_file_path.endswith('/'):
                    print(f"Skipping folder as it is empty: {s3_file_path}")
                    continue

                # Normalize path for Windows by replacing '/' with the OS-specific separator
                local_file_path = os.path.join(local_dir, s3_file_path.replace('/', os.sep))

                # Ensure local directory structure matches S3 structure
                local_file_dir = os.path.dirname(local_file_path)
                if not os.path.exists(local_file_dir):
                    try:
                        os.makedirs(local_file_dir)
                    except OSError as e:
                        print(f"Error creating directory {local_file_dir}: {e}")
                        continue  # Skip this file if directory creation fails

                print(f"Downloading {s3_file_path} to {local_file_path}")
                
                # Download the file
                try:
                    s3.download_file(bucket_name, s3_file_path, local_file_path)
                except Exception as e:
                    print(f"Error downloading {s3_file_path}: {e}")


# Usage example
if __name__ == "__main__":
    bucket_name = 'mydev-storage'  # S3 bucket name
    local_dir = r'D:\\aws_s3_downloads'  # Local directory where files will be saved

    download_s3_bucket(bucket_name, local_dir)
